# 데이터 탐색 및 DB 연결 테스트

이 노트북은 가격 예측 프로젝트의 데이터 탐색과 데이터베이스 연결을 테스트합니다.

## 1. 라이브러리 임포트

In [2]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

## 2. 데이터베이스 연결 테스트

In [3]:
from src.data.db_connector import DatabaseConnector, SessionManager

# 데이터베이스 연결 테스트
print("=" * 50)
print("데이터베이스 연결 테스트")
print("=" * 50)

try:
    with DatabaseConnector() as db:
        print("✓ 데이터베이스 연결 성공!\n")
        
        # 테이블 목록 조회
        tables = db.get_table_list()
        print(f"✓ 사용 가능한 테이블 수: {len(tables)}")
        print("\n테이블 목록:")
        for i, table in enumerate(tables[:10], 1):
            print(f"  {i}. {table}")
        
        if len(tables) > 10:
            print(f"  ... 외 {len(tables) - 10}개")
        
        print("\n" + "=" * 50)
        print("연결 테스트 완료")
        print("=" * 50)
        
except Exception as e:
    print(f"✗ 연결 실패: {e}")

데이터베이스 연결 테스트


INFO:src.data.db_connector:Successfully connected to database: qai


✓ 데이터베이스 연결 성공!



INFO:src.data.db_connector:Database connection pool disposed


✓ 사용 가능한 테이블 수: 0

테이블 목록:

연결 테스트 완료


## 3. Step 1: 종목 필터링 (마스터 리스트 생성)

목표: NYSE, NASDAQ, AMEX 종목 중 조건을 만족하는 종목 추출

In [4]:
# ORM 모델 임포트
from src.data.models import (
    DS2CtryQtInfo, 
    DS2Exchange, 
    DS2PrimQtPrc,
    VwDs2Pricing,
    VwDs2MktCap,
    RKDFndInfo  # Ticker, ISIN 정보를 위해 추가
)
from sqlalchemy import func, and_, or_
from sqlalchemy.orm import aliased

print("✓ ORM 모델 임포트 완료")

✓ ORM 모델 임포트 완료


In [5]:
# Step 1: NYSE, NASDAQ, AMEX 종목 리스트 조회 (ORM 사용 + Ticker/ISIN + 거래소명)
print("=" * 80)
print("Step 1: 종목 리스트 조회 (ORM) with Ticker/ISIN/Exchange")
print("=" * 80)

try:
    with DatabaseConnector() as db:
        with SessionManager(db) as session:
            # 1. NYSE, NASDAQ, AMEX 거래소 필터링
            print("\n[1] NYSE, NASDAQ, AMEX 필터링...")
            target_exchanges = session.query(DS2Exchange).filter(
                and_(
                    DS2Exchange.ExchCtryCode == 'US',
                    or_(
                        DS2Exchange.ExchName.like('%NYSE%'),
                        DS2Exchange.ExchName.like('%NASDAQ%'),
                        DS2Exchange.ExchName.like('%AMEX%')
                    )
                )
            ).all()
            
            print(f"✓ 대상 거래소:")
            for exch in target_exchanges[:5]:
                print(f"  - {exch.ExchName} (코드: {exch.ExchIntCode})")
            if len(target_exchanges) > 5:
                print(f"  ... 외 {len(target_exchanges) - 5}개")
            
            target_exch_codes = [exch.ExchIntCode for exch in target_exchanges]
            
            # 2. 종목 조회 (거래소, Ticker/ISIN 정보 포함)
            print(f"\n[2] 종목 조회 중...")
            
            # DS2PrimQtPrc에서 거래소별 종목 조회 후 DS2Exchange 조인
            stocks = session.query(
                DS2CtryQtInfo.InfoCode,
                DS2CtryQtInfo.DsCode,
                DS2CtryQtInfo.DsQtName,
                DS2CtryQtInfo.StatusCode,
                DS2CtryQtInfo.DelistDate,
                DS2PrimQtPrc.ExchIntCode,
                DS2Exchange.ExchName,
                DS2Exchange.ExchMnem,
                RKDFndInfo.Ticker,
                RKDFndInfo.ISIN,
                RKDFndInfo.Cusip,
                RKDFndInfo.Sedol,
            ).join(
                DS2PrimQtPrc,
                DS2CtryQtInfo.InfoCode == DS2PrimQtPrc.InfoCode
            ).join(
                DS2Exchange,
                DS2PrimQtPrc.ExchIntCode == DS2Exchange.ExchIntCode
            ).outerjoin(
                RKDFndInfo, 
                DS2CtryQtInfo.InfoCode == RKDFndInfo.Code
            ).filter(
                and_(
                    DS2CtryQtInfo.Region == 'US',
                    DS2CtryQtInfo.IsPrimQt == 1,
                    DS2PrimQtPrc.ExchIntCode.in_(target_exch_codes)
                )
            ).distinct().limit(100).all()
            
            print(f"✓ 조회된 종목 수: {len(stocks)} (샘플 100개)")
            
            # DataFrame으로 변환
            df_stocks = pd.DataFrame([
                {
                    'InfoCode': s.InfoCode,
                    'DsCode': s.DsCode,
                    'DsQtName': s.DsQtName,
                    'Exchange': s.ExchName,
                    'ExchMnem': s.ExchMnem,
                    'ExchIntCode': s.ExchIntCode,
                    'Ticker': s.Ticker,
                    'ISIN': s.ISIN,
                    'Cusip': s.Cusip,
                    'Sedol': s.Sedol,
                    'StatusCode': s.StatusCode,
                    'DelistDate': s.DelistDate,
                }
                for s in stocks
            ])
            
            print(f"\n✓ 데이터프레임 생성 완료:")
            print(f"  - Shape: {df_stocks.shape}")
            print(f"  - Ticker 매핑률: {df_stocks['Ticker'].notna().sum()}/{len(df_stocks)} ({df_stocks['Ticker'].notna().sum()/len(df_stocks)*100:.1f}%)")
            print(f"  - ISIN 매핑률: {df_stocks['ISIN'].notna().sum()}/{len(df_stocks)} ({df_stocks['ISIN'].notna().sum()/len(df_stocks)*100:.1f}%)")
            print(f"  - 상장폐지 종목: {df_stocks['DelistDate'].notna().sum()}")
            
            print(f"\n✓ 거래소별 분포:")
            print(df_stocks['Exchange'].value_counts())
            
            print(f"\n샘플 데이터 (처음 10개):")
            print(df_stocks[['DsCode', 'DsQtName', 'Exchange', 'Ticker', 'ISIN']].head(10))
            
except Exception as e:
    print(f"✗ 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    df_stocks = pd.DataFrame()

Step 1: 종목 리스트 조회 (ORM) with Ticker/ISIN/Exchange


INFO:src.data.db_connector:Successfully connected to database: qai



[1] NYSE, NASDAQ, AMEX 필터링...
✓ 대상 거래소:
  - NYSE American (코드: 12)
  - Nasdaq (코드: 135)
  - Nasdaq Smallcap (코드: 140)
  - Nasdaq Interlisted (코드: 195)
  - NYSE ARCA (코드: 244)
  ... 외 14개

[2] 종목 조회 중...


INFO:src.data.db_connector:Database connection pool disposed


✓ 조회된 종목 수: 100 (샘플 100개)

✓ 데이터프레임 생성 완료:
  - Shape: (100, 12)
  - Ticker 매핑률: 70/100 (70.0%)
  - ISIN 매핑률: 74/100 (74.0%)
  - 상장폐지 종목: 62

✓ 거래소별 분포:
Exchange
Nasdaq           81
NYSE American    17
NYSE ARCA         2
Name: count, dtype: int64

샘플 데이터 (처음 10개):
   DsCode                                    DsQtName       Exchange  \
0  542868                               CISCO SYSTEMS         Nasdaq   
1  873886               ALTABA DEAD - DELIST.12/10/19         Nasdaq   
2  922726                                       INTEL         Nasdaq   
3  992017                               HELEN OF TROY         Nasdaq   
4  326797       ELTN.FOR IMAG. DEAD - DELIST.23/07/19         Nasdaq   
5  29408L                     CALAMOS GLB.TTL.RTN.FD.         Nasdaq   
6  867076         PHARMACYCLICS DEAD - ACQD.BY 87851X         Nasdaq   
7  27947E    CALLIDUS SOFTWARE DEAD - DELIST.06/04/18         Nasdaq   
8  677382                  COMPX INTERNATIONAL(ASE) A  NYSE American   
9  326996  PATT

## 4. Step 2: 갭 상승 데이터 추출

목표: 거래일 기준으로 갭 상승 이벤트 찾기
- 갭 상승률 = (당일 시가 - 전일 종가) / 전일 종가 × 100

In [6]:
# Step 2-1: 샘플 종목의 일별 가격 데이터 조회 및 갭 상승 계산
print("=" * 80)
print("Step 2: 갭 상승 데이터 추출")
print("=" * 80)

if not df_stocks.empty:
    # 샘플 종목 선택 (상장 중인 종목 중 5개)
    sample_stocks = df_stocks[df_stocks['DelistDate'].isna()].head(5)
    sample_info_codes = sample_stocks['InfoCode'].tolist()
    
    print(f"\n샘플 종목 ({len(sample_info_codes)}개):")
    for _, stock in sample_stocks.iterrows():
        print(f"  - {stock['DsQtName']} ({stock['Ticker']}) / {stock['Exchange']}")
    
    try:
        with DatabaseConnector() as db:
            with SessionManager(db) as session:
                print(f"\n일별 가격 데이터 조회 중...")
                
                # 일별 OHLCV 데이터 조회 (최근 2년)
                from datetime import datetime, timedelta
                start_date = datetime.now() - timedelta(days=730)  # 2년
                
                prices = session.query(
                    DS2PrimQtPrc.InfoCode,
                    DS2PrimQtPrc.MarketDate,
                    DS2PrimQtPrc.Open_,
                    DS2PrimQtPrc.High,
                    DS2PrimQtPrc.Low,
                    DS2PrimQtPrc.Close_,
                    DS2PrimQtPrc.Volume,
                ).filter(
                    and_(
                        DS2PrimQtPrc.InfoCode.in_(sample_info_codes),
                        DS2PrimQtPrc.MarketDate >= start_date,
                        DS2PrimQtPrc.Open_.isnot(None),
                        DS2PrimQtPrc.Close_.isnot(None)
                    )
                ).order_by(
                    DS2PrimQtPrc.InfoCode,
                    DS2PrimQtPrc.MarketDate
                ).all()
                
                print(f"✓ 조회된 레코드 수: {len(prices):,}")
                
                # DataFrame으로 변환
                df_prices = pd.DataFrame([
                    {
                        'InfoCode': p.InfoCode,
                        'MarketDate': p.MarketDate,
                        'Open': p.Open_,
                        'High': p.High,
                        'Low': p.Low,
                        'Close': p.Close_,
                        'Volume': p.Volume,
                    }
                    for p in prices
                ])
                
                # 종목별로 전일 종가 계산 및 갭 계산
                print(f"\n갭 상승 계산 중...")
                df_prices = df_prices.sort_values(['InfoCode', 'MarketDate'])
                df_prices['PrevClose'] = df_prices.groupby('InfoCode')['Close'].shift(1)
                df_prices['Gap_Pct'] = ((df_prices['Open'] - df_prices['PrevClose']) / df_prices['PrevClose'] * 100)
                df_prices['Intraday_Return'] = ((df_prices['Close'] - df_prices['Open']) / df_prices['Open'] * 100)
                
                # 갭 상승 필터링 (갭 > 0%)
                df_gap_up = df_prices[df_prices['Gap_Pct'] > 0].copy()
                df_gap_up['Target'] = (df_gap_up['Intraday_Return'] > 0).astype(int)
                
                print(f"✓ 갭 상승 이벤트 수: {len(df_gap_up):,}")
                print(f"✓ 전체 거래일 대비 비율: {len(df_gap_up)/len(df_prices)*100:.1f}%")
                
                # 종목 정보 매핑
                df_gap_up = df_gap_up.merge(
                    df_stocks[['InfoCode', 'DsQtName', 'Ticker', 'Exchange']],
                    on='InfoCode',
                    how='left'
                )
                
                print(f"\n✓ 갭 상승 통계:")
                print(f"  - 평균 갭 상승률: {df_gap_up['Gap_Pct'].mean():.2f}%")
                print(f"  - 중간 갭 상승률: {df_gap_up['Gap_Pct'].median():.2f}%")
                print(f"  - 최대 갭 상승률: {df_gap_up['Gap_Pct'].max():.2f}%")
                print(f"  - 당일 상승 비율 (Target=1): {df_gap_up['Target'].mean()*100:.1f}%")
                
                print(f"\n샘플 갭 상승 데이터 (최근 10개):")
                print(df_gap_up[['MarketDate', 'DsQtName', 'Ticker', 'Gap_Pct', 'Intraday_Return', 'Target']].tail(10))
                
    except Exception as e:
        print(f"✗ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        df_gap_up = pd.DataFrame()
else:
    print("\n✗ 종목 데이터가 없습니다.")
    df_gap_up = pd.DataFrame()

Step 2: 갭 상승 데이터 추출

샘플 종목 (5개):
  - CISCO SYSTEMS (5607) / Nasdaq
  - INTEL (532) / Nasdaq
  - HELEN OF TROY (None) / Nasdaq
  - CALAMOS GLB.TTL.RTN.FD. (EWII) / Nasdaq
  - COMPX INTERNATIONAL(ASE) A (DKIEREUAE) / NYSE American


INFO:src.data.db_connector:Successfully connected to database: qai
INFO:src.data.db_connector:Database connection pool disposed



일별 가격 데이터 조회 중...
✓ 조회된 레코드 수: 2,500

갭 상승 계산 중...
✓ 갭 상승 이벤트 수: 1,270
✓ 전체 거래일 대비 비율: 50.8%

✓ 갭 상승 통계:
  - 평균 갭 상승률: 1.02%
  - 중간 갭 상승률: 0.58%
  - 최대 갭 상승률: 27.63%
  - 당일 상승 비율 (Target=1): 46.0%

샘플 갭 상승 데이터 (최근 10개):
     MarketDate                    DsQtName     Ticker   Gap_Pct  \
1260 2025-12-05  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  1.724138   
1261 2025-12-09  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  2.083319   
1262 2025-12-11  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  0.049335   
1263 2025-12-12  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  0.378439   
1264 2025-12-15  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  2.566438   
1265 2025-12-18  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  0.205520   
1266 2025-12-26  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  2.390474   
1267 2025-12-30  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  0.173355   
1268 2025-12-31  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  0.840650   
1269 2026-01-05  COMPX INTERNATIONAL(ASE) A  DKIEREUAE  1.760865   

      Intraday_Return  Target 

## 5. Step 3: 전체 파이프라인 테스트

**목표:** 
- DataLoader, DataPreprocessor, Feature Engineering 모듈 통합 테스트
- 소수 종목으로 빠르게 검증 후 전체 종목 적용

In [13]:
# 🔍 디버깅: Ticker/ISIN이 어디서 사라지는지 확인
print("="*80)
print("디버깅: Ticker/ISIN 추적")
print("="*80)

print("\n1. stock_list_full 확인:")
if 'stock_list_full' in locals():
    print(f"   컬럼: {list(stock_list_full.columns)}")
    print(f"   Ticker 있음: {'Ticker' in stock_list_full.columns}")
    print(f"   ISIN 있음: {'ISIN' in stock_list_full.columns}")
    if 'Ticker' in stock_list_full.columns:
        print(f"   Ticker 샘플: {stock_list_full['Ticker'].head(3).tolist()}")

print("\n2. sample_stocks 확인:")
if 'sample_stocks' in locals():
    print(f"   컬럼: {list(sample_stocks.columns)}")
    print(f"   Ticker 있음: {'Ticker' in sample_stocks.columns}")

print("\n3. sample_stocks_with_sector 확인:")
if 'sample_stocks_with_sector' in locals():
    print(f"   컬럼: {list(sample_stocks_with_sector.columns)}")
    print(f"   Ticker 있음: {'Ticker' in sample_stocks_with_sector.columns}")
    print(f"   ISIN 있음: {'ISIN' in sample_stocks_with_sector.columns}")
    if 'Ticker' in sample_stocks_with_sector.columns:
        print(f"   Ticker 샘플: {sample_stocks_with_sector['Ticker'].head(3).tolist()}")

print("\n4. df_preprocessed 확인:")
if 'df_preprocessed' in locals():
    print(f"   전체 컬럼 수: {len(df_preprocessed.columns)}")
    print(f"   Ticker 있음: {'Ticker' in df_preprocessed.columns}")
    print(f"   ISIN 있음: {'ISIN' in df_preprocessed.columns}")
    
    # 중복 컬럼 확인 (Ticker_x, Ticker_y 같은 것)
    duplicate_cols = [c for c in df_preprocessed.columns if '_x' in c or '_y' in c]
    if duplicate_cols:
        print(f"   ⚠️ 중복 컬럼 발견: {duplicate_cols}")

print("\n5. df_final 확인:")
if 'df_final' in locals():
    print(f"   전체 컬럼 수: {len(df_final.columns)}")
    print(f"   Ticker 있음: {'Ticker' in df_final.columns}")
    print(f"   ISIN 있음: {'ISIN' in df_final.columns}")
    print(f"   sector 있음: {'sector' in df_final.columns}")
    print(f"   spy_gap_pct 있음: {'spy_gap_pct' in df_final.columns}")
    
    # 실제로 있는 관련 컬럼들 출력
    related_cols = [c for c in df_final.columns if any(x in c.lower() for x in ['ticker', 'isin', 'sector', 'spy', 'qqq', 'vix'])]
    print(f"   관련 컬럼: {related_cols}")

디버깅: Ticker/ISIN 추적

1. stock_list_full 확인:
   컬럼: ['InfoCode', 'DsCode', 'DsQtName', 'Exchange', 'ExchMnem', 'ExchIntCode', 'Ticker', 'ISIN', 'Cusip', 'Sedol', 'StatusCode', 'DelistDate', 'Region']
   Ticker 있음: True
   ISIN 있음: True
   Ticker 샘플: ['IMND', None, None]

2. sample_stocks 확인:
   컬럼: ['InfoCode', 'DsCode', 'DsQtName', 'Exchange', 'ExchMnem', 'ExchIntCode', 'Ticker', 'ISIN', 'Cusip', 'Sedol', 'StatusCode', 'DelistDate', 'Region']
   Ticker 있음: True

3. sample_stocks_with_sector 확인:
   컬럼: ['InfoCode', 'DsCode', 'DsQtName', 'Exchange', 'ExchMnem', 'ExchIntCode', 'Ticker', 'ISIN', 'Cusip', 'Sedol', 'StatusCode', 'DelistDate', 'Region', 'sector', 'industry']
   Ticker 있음: True
   ISIN 있음: True
   Ticker 샘플: [None, None, None]

4. df_preprocessed 확인:
   전체 컬럼 수: 44
   Ticker 있음: True
   ISIN 있음: True

5. df_final 확인:
   전체 컬럼 수: 70
   Ticker 있음: True
   ISIN 있음: True
   sector 있음: True
   spy_gap_pct 있음: True
   관련 컬럼: ['Ticker', 'ISIN', 'sector', 'spy_gap_pct', 'qqq_gap_pct',

### 🔍 디버깅: 데이터 확인

### 5-1. 모듈 임포트

In [ ]:
pip install tqdm

In [1]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 전체 파이프라인 모듈 임포트
from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor
from src.features import add_technical_features_parallel, add_market_context

print("✓ 전체 파이프라인 모듈 임포트 완료")

✓ 전체 파이프라인 모듈 임포트 완료


### 5-2. 데이터 로더 초기화 및 샘플 데이터 로드

**테스트 전략:**
- 먼저 10개 종목으로 빠르게 테스트
- 문제 없으면 전체 종목으로 확장

In [2]:
# Step 1: DataLoader 초기화
print("="*80)
print("Step 1: DataLoader 초기화")
print("="*80)

loader = DataLoader(config_path="../config/config.yaml")
print(f"✓ Config loaded: {loader.config.get('data', {}).get('min_years', 10)} years")
print(f"✓ Batch size: {loader.config.get('data', {}).get('parallel', {}).get('batch_size', 100)}")

Step 1: DataLoader 초기화
✓ Config loaded: 10 years
✓ Batch size: 100


In [3]:
# Step 2: 샘플 종목 리스트 조회 (10개)
print("\n" + "="*80)
print("Step 2: 샘플 종목 리스트 조회")
print("="*80)

# 전체 종목 리스트 로드
stock_list_full = loader.get_stock_list()
print(f"✓ 전체 종목 수: {len(stock_list_full)}")

# 샘플 10개 선택 (상장 중인 종목 우선)
sample_stocks = stock_list_full[stock_list_full['DelistDate'].isna()].head(10)
print(f"✓ 샘플 종목 수: {len(sample_stocks)}")

print("\n샘플 종목 목록:")
for idx, row in sample_stocks.iterrows():
    print(f"  {row['InfoCode']}: {row['DsQtName']} ({row['Ticker']}) - {row['Exchange']}")


Step 2: 샘플 종목 리스트 조회


INFO:src.data.db_connector:Successfully connected to database: qai


✓ 전체 종목 수: 19996
✓ 샘플 종목 수: 10

샘플 종목 목록:
  374605: KSHS.SUST.ULTRA SHT.DUR. IDX.ETF (None) - NYSE ARCA
  385651: OXLEY BRIDGE ACQUISITION A (None) - Nasdaq
  258636: STE.STR.SPDR PRTF. SHTM. TRSY.ETF (None) - NYSE ARCA
  153466: DUOS TECHNOLOGIES GROUP (None) - Nasdaq
  54849: TRACTOR SUPPLY (PIA) - Nasdaq
  328927: CUREVAC (None) - Nasdaq
  213226: ISHARES EMM.INFR.ETF (None) - Nasdaq
  319736: INSPIRE INTERNATIONAL ETF (None) - NYSE ARCA
  170228: DIREXION DAILY SMALL CAP BULL 3X SHARES (None) - NYSE ARCA
  73780: FIRST TST.CAP.STRENGHT ETF (None) - Nasdaq


In [4]:
# Step 3: 섹터/산업 정보 로드
print("\n" + "="*80)
print("Step 3: 섹터/산업 정보 로드")
print("="*80)

sample_info_codes = sample_stocks['InfoCode'].tolist()
sector_info = loader.load_sector_industry(sample_info_codes)

# 샘플 종목과 병합
sample_stocks_with_sector = sample_stocks.merge(sector_info, on='InfoCode', how='left')

print(f"✓ 섹터 정보 조회 완료")
print(f"  - 섹터 매핑률: {sample_stocks_with_sector['sector'].notna().sum()}/{len(sample_stocks_with_sector)}")
print(f"  - 산업 매핑률: {sample_stocks_with_sector['industry'].notna().sum()}/{len(sample_stocks_with_sector)}")

print("\n섹터/산업 정보:")
print(sample_stocks_with_sector[['DsQtName', 'sector', 'industry']].head(10))


Step 3: 섹터/산업 정보 로드
✓ 섹터 정보 조회 완료
  - 섹터 매핑률: 1/10
  - 산업 매핑률: 1/10

섹터/산업 정보:
                                  DsQtName     sector  \
0         KSHS.SUST.ULTRA SHT.DUR. IDX.ETF        NaN   
1               OXLEY BRIDGE ACQUISITION A        NaN   
2        STE.STR.SPDR PRTF. SHTM. TRSY.ETF        NaN   
3                  DUOS TECHNOLOGIES GROUP        NaN   
4                           TRACTOR SUPPLY  Financial   
5                                  CUREVAC        NaN   
6                     ISHARES EMM.INFR.ETF        NaN   
7                INSPIRE INTERNATIONAL ETF        NaN   
8  DIREXION DAILY SMALL CAP BULL 3X SHARES        NaN   
9               FIRST TST.CAP.STRENGHT ETF        NaN   

                   industry  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4  Other Financial Vehicles  
5                       NaN  
6                       NaN  
7                       NaN  
8                     

In [5]:
# Step 4: OHLCV 데이터 로드 (최근 2년)
print("\n" + "="*80)
print("Step 4: OHLCV 데이터 로드")
print("="*80)

from datetime import datetime, timedelta

start_date = datetime.now() - timedelta(days=365 * 2)  # 2년
print(f"기간: {start_date.date()} ~ {datetime.now().date()}")

# 병렬로 OHLCV 로드
ohlcv_sample = loader.load_ohlcv_batch(sample_info_codes, start_date, adj_type=2)

print(f"\n✓ OHLCV 데이터 로드 완료")
print(f"  - 총 레코드 수: {len(ohlcv_sample):,}")
print(f"  - 종목 수: {ohlcv_sample['InfoCode'].nunique()}")
print(f"  - 날짜 범위: {ohlcv_sample['date'].min()} ~ {ohlcv_sample['date'].max()}")

print("\n샘플 데이터:")
print(ohlcv_sample.head())


Step 4: OHLCV 데이터 로드
기간: 2024-01-08 ~ 2026-01-07


Loading OHLCV:   0%|          | 0/10 [00:00<?, ?it/s]

Loading OHLCV: 100%|██████████| 10/10 [00:01<00:00,  5.19it/s]


✓ OHLCV 데이터 로드 완료
  - 총 레코드 수: 70,282
  - 종목 수: 10
  - 날짜 범위: 2024-01-09 00:00:00 ~ 2026-01-06 00:00:00

샘플 데이터:
   InfoCode       date       open       high        low      close     volume  \
0    374605 2024-07-26  25.070008  25.080002  25.070008  25.080002  3992374.0   
1    374605 2024-07-26  25.070008  25.080002  25.070008  25.080002     1131.0   
2    374605 2024-07-26  25.070008  25.080002  25.070008  25.080002  3992374.0   
3    374605 2024-07-26  25.070008  25.080002  25.070008  25.080002  3992374.0   
4    374605 2024-07-26  25.070008  25.070008  25.070008  25.070008  3991243.0   

   ExchIntCode  
0          244  
1          253  
2          348  
3          355  
4          395  


In [6]:
# Step 5: 시장 지수 데이터 로드 (SPY, QQQ, VIX)
print("\n" + "="*80)
print("Step 5: 시장 지수 데이터 로드")
print("="*80)

market_indices = loader.load_market_indices(start_date)

if not market_indices.empty:
    print(f"✓ 시장 지수 데이터 로드 완료")
    print(f"  - 레코드 수: {len(market_indices):,}")
    print(f"  - 컬럼: {list(market_indices.columns)}")
    print("\n샘플 데이터:")
    print(market_indices.head())
else:
    print("✗ 시장 지수 데이터 없음")


Step 5: 시장 지수 데이터 로드

Loading market indices...
   Loading SPY (S&PCOMP)...
   ✓ Loaded 501 records
   Loading QQQ (NASD100)...
   ✗ No data found
   Loading VIX (CBOE)...
   ✗ No data found
✓ Merged 501 date records
✓ 시장 지수 데이터 로드 완료
  - 레코드 수: 501
  - 컬럼: ['date', 'spy_pi', 'spy_ri']

샘플 데이터:
        date     spy_pi     spy_ri
0 2024-01-09  4756.5000  10302.731
1 2024-01-10  4783.4493  10361.270
2 2024-01-11  4780.2383  10354.692
3 2024-01-12  4783.8282  10363.352
4 2024-01-16  4765.9805  10324.829


### 5-3. 데이터 전처리 (Preprocessor)

In [7]:
# Step 6: 데이터 전처리 (병렬 처리)
print("\n" + "="*80)
print("Step 6: 데이터 전처리")
print("="*80)

# Preprocessor 초기화
preprocessor = DataPreprocessor(loader.config)

# preprocessed_df 생성 (병렬 처리 사용)
df_preprocessed = preprocessor.create_preprocessed_df(
    ohlcv_df=ohlcv_sample,
    stock_list=sample_stocks_with_sector,
    market_cap_df=None,  # 시가총액은 생략 (빠른 테스트용)
    use_parallel=True
)

print(f"\n✓ 전처리 완료")
print(f"  - 최종 Shape: {df_preprocessed.shape}")
print(f"  - 갭 상승 이벤트: {df_preprocessed['is_gap_up'].sum():,}")


Step 6: 데이터 전처리

=== 데이터 전처리 시작 ===
1-5. Processing features in parallel (lag, gap, returns, volume, targets)...
   Processing 10 stocks with 10 workers...


   Processing stocks: 100%|██████████| 10/10 [00:00<00:00, 22.95it/s]

   ✓ Shape: (70282, 25)
   ✓ Gap up events: 33,052
6. Adding time features...
   ✓ Added: day_of_week, month, quarter, is_month_start, is_month_end, is_quarter_end
7. Skipping market cap (no data)
8. Merging stock info...
9. Applying filters...
   ✓ Filtered: 70,282 -> 33,599 (47.8%)
10. Organizing columns...

=== 전처리 완료 ===
Final shape: (33599, 44)
Date range: 2024-01-09 00:00:00 ~ 2026-01-06 00:00:00
Unique stocks: 10

✓ 전처리 완료
  - 최종 Shape: (33599, 44)
  - 갭 상승 이벤트: 15,813


### 5-4. Feature Engineering (기술적 지표 + 시장 컨텍스트)

In [8]:
# Step 7: 기술적 지표 추가 (병렬 처리)
print("\n" + "="*80)
print("Step 7: 기술적 지표 추가")
print("="*80)

df_with_technical = add_technical_features_parallel(df_preprocessed, max_workers=4)

print(f"\n✓ 기술적 지표 추가 완료")
print(f"  - Shape: {df_with_technical.shape}")

# 추가된 컬럼 확인
technical_cols = ['ma_5', 'ma_20', 'ma_50', 'rsi_14', 'atr_14', 'bollinger_position']
print(f"\n기술적 지표 샘플:")
print(df_with_technical[['date', 'DsQtName', 'close'] + technical_cols].tail(5))


Step 7: 기술적 지표 추가
   Adding technical features to 10 stocks with 4 workers...


   Technical indicators: 100%|██████████| 10/10 [00:00<00:00, 36.49it/s]


✓ 기술적 지표 추가 완료
  - Shape: (33599, 62)

기술적 지표 샘플:
            date                    DsQtName      close       ma_5      ma_20  \
33594 2025-12-18  OXLEY BRIDGE ACQUISITION A  10.019801  10.019961  10.028991   
33595 2025-12-18  OXLEY BRIDGE ACQUISITION A  10.020001  10.019961  10.026491   
33596 2025-12-18  OXLEY BRIDGE ACQUISITION A  10.020001  10.019961  10.024491   
33597 2025-12-18  OXLEY BRIDGE ACQUISITION A  10.020001  10.019961  10.022491   
33598 2025-12-18  OXLEY BRIDGE ACQUISITION A  10.020001  10.019961  10.020491   

           ma_50     rsi_14    atr_14  bollinger_position  
33594  10.036297  41.921459  0.005851            0.352227  
33595  10.036097  41.776822  0.005447            0.368317  
33596  10.035897  41.992355  0.005072            0.388975  
33597  10.035697  41.992355  0.004724            0.409093  
33598  10.035497  41.992355  0.004387            0.431652  


In [9]:
# Step 8: 시장 컨텍스트 추가
print("\n" + "="*80)
print("Step 8: 시장 컨텍스트 추가")
print("="*80)

df_final = add_market_context(df_with_technical, market_indices)

print(f"\n✓ 시장 컨텍스트 추가 완료")
print(f"  - 최종 Shape: {df_final.shape}")

# 시장 컨텍스트 컬럼 확인
market_cols = ['spy_gap_pct', 'qqq_gap_pct', 'market_gap_diff', 'vix_level', 
               'sector', 'sector_gap_pct', 'sector_relative_gap']
print(f"\n시장 컨텍스트 샘플:")
print(df_final[['date', 'DsQtName', 'gap_pct'] + market_cols].tail(5))


Step 8: 시장 컨텍스트 추가
   Calculating market index gaps...
   Merging market features...
   ✓ Market data coverage: 99.8%

✓ 시장 컨텍스트 추가 완료
  - 최종 Shape: (33599, 70)

시장 컨텍스트 샘플:
            date                    DsQtName   gap_pct  spy_gap_pct  \
33594 2025-12-18  OXLEY BRIDGE ACQUISITION A  0.000000     0.793464   
33595 2025-12-18  OXLEY BRIDGE ACQUISITION A  0.001996     0.793464   
33596 2025-12-18  OXLEY BRIDGE ACQUISITION A  0.000000     0.793464   
33597 2025-12-18  OXLEY BRIDGE ACQUISITION A  0.000000     0.793464   
33598 2025-12-18  OXLEY BRIDGE ACQUISITION A  0.000000     0.793464   

       qqq_gap_pct  market_gap_diff  vix_level sector  sector_gap_pct  \
33594          NaN        -0.793464        NaN    NaN             NaN   
33595          NaN        -0.791468        NaN    NaN             NaN   
33596          NaN        -0.793464        NaN    NaN             NaN   
33597          NaN        -0.793464        NaN    NaN             NaN   
33598          NaN        -0.7934

### 5-5. 최종 결과 확인 및 통계

In [11]:
# 최종 결과 확인
print("="*80)
print("최종 데이터프레임 정보")
print("="*80)

print(f"\n1. 기본 정보")
print(f"   - Shape: {df_final.shape}")
print(f"   - 종목 수: {df_final['InfoCode'].nunique()}")
print(f"   - 날짜 범위: {df_final['date'].min()} ~ {df_final['date'].max()}")
print(f"   - 메모리 사용량: {df_final.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print(f"\n2. 컬럼 구성 ({len(df_final.columns)}개)")
print(f"   - 전체 컬럼: {list(df_final.columns[:20])}...")

print(f"\n3. 갭 상승 이벤트 통계")
gap_up_df = df_final[df_final['is_gap_up'] == 1]
print(f"   - 갭 상승 이벤트 수: {len(gap_up_df):,}")
print(f"   - 평균 갭 상승률: {gap_up_df['gap_pct'].mean():.2f}%")
print(f"   - 당일 상승 비율: {gap_up_df['target_direction'].mean()*100:.1f}%")

print(f"\n4. Feature 커버리지")
print(f"   - 섹터 정보: {df_final['sector'].notna().sum()}/{len(df_final)} ({df_final['sector'].notna().mean()*100:.1f}%)")
print(f"   - 산업 정보: {df_final['industry'].notna().sum()}/{len(df_final)} ({df_final['industry'].notna().mean()*100:.1f}%)")
print(f"   - RSI 14: {df_final['rsi_14'].notna().sum()}/{len(df_final)} ({df_final['rsi_14'].notna().mean()*100:.1f}%)")
print(f"   - SPY 갭: {df_final['spy_gap_pct'].notna().sum()}/{len(df_final)} ({df_final['spy_gap_pct'].notna().mean()*100:.1f}%)")

print(f"\n5. 샘플 데이터 (갭 상승 이벤트만)")
display_cols = ['date', 'DsQtName', 'sector', 'gap_pct', 'target_direction', 
                'rsi_14', 'spy_gap_pct', 'market_gap_diff']
print(gap_up_df[display_cols].tail(10))

최종 데이터프레임 정보

1. 기본 정보
   - Shape: (3259, 70)
   - 종목 수: 6
   - 날짜 범위: 2024-02-23 00:00:00 ~ 2026-01-06 00:00:00
   - 메모리 사용량: 2.74 MB

2. 컬럼 구성 (70개)
   - 전체 컬럼: ['InfoCode', 'DsCode', 'date', 'Ticker', 'ISIN', 'DsQtName', 'Exchange', 'ExchMnem', 'Region', 'open', 'high', 'low', 'close', 'volume', 'prev_open', 'prev_high', 'prev_low', 'prev_close', 'prev_volume', 'gap_pct']...

3. 갭 상승 이벤트 통계
   - 갭 상승 이벤트 수: 1,518
   - 평균 갭 상승률: 3.82%
   - 당일 상승 비율: 7.0%

4. Feature 커버리지
   - 섹터 정보: 0/3259 (0.0%)
   - 산업 정보: 0/3259 (0.0%)
   - RSI 14: 3188/3259 (97.8%)
   - SPY 갭: 3259/3259 (100.0%)

5. 샘플 데이터 (갭 상승 이벤트만)
           date                                DsQtName  sector    gap_pct  \
3246 2025-09-23     HERITAGE DISTILLING HOLDING COMPANY     NaN   5.681819   
3247 2025-09-23     HERITAGE DISTILLING HOLDING COMPANY     NaN  14.005516   
3248 2025-09-23     HERITAGE DISTILLING HOLDING COMPANY     NaN  13.998527   
3249 2025-09-23     HERITAGE DISTILLING HOLDING COMPANY     NaN  25.97403

In [10]:
# 디버깅: 각 단계별 데이터 확인
print("="*80)
print("디버깅: 데이터 확인")
print("="*80)

print("\n1. sample_stocks_with_sector 컬럼:")
print(f"   {list(sample_stocks_with_sector.columns)}")
print(f"   sector 있음: {'sector' in sample_stocks_with_sector.columns}")
print(f"   industry 있음: {'industry' in sample_stocks_with_sector.columns}")

print("\n2. market_indices 상태:")
if 'market_indices' in locals() and market_indices is not None:
    print(f"   비어있음: {market_indices.empty}")
    if not market_indices.empty:
        print(f"   컬럼: {list(market_indices.columns)}")
        print(f"   레코드 수: {len(market_indices)}")
    else:
        print("   ⚠️ market_indices가 비어있습니다!")
else:
    print("   ⚠️ market_indices가 정의되지 않았습니다!")

print("\n3. df_preprocessed 컬럼 (샘플 20개):")
if 'df_preprocessed' in locals():
    print(f"   {list(df_preprocessed.columns[:20])}...")
    print(f"   Ticker 있음: {'Ticker' in df_preprocessed.columns}")
    print(f"   sector 있음: {'sector' in df_preprocessed.columns}")
else:
    print("   ⚠️ df_preprocessed가 아직 생성되지 않았습니다!")

print("\n4. df_final 컬럼 (샘플 20개):")
if 'df_final' in locals():
    print(f"   전체 컬럼 수: {len(df_final.columns)}")
    print(f"   {list(df_final.columns[:20])}...")
    
    # 누락된 컬럼 확인
    expected_cols = ['Ticker', 'ISIN', 'sector', 'industry', 
                     'spy_gap_pct', 'qqq_gap_pct', 'vix_level', 
                     'sector_gap_pct', 'sector_relative_gap']
    missing = [c for c in expected_cols if c not in df_final.columns]
    if missing:
        print(f"\n   ⚠️ 누락된 컬럼: {missing}")
    else:
        print(f"\n   ✓ 모든 예상 컬럼이 존재합니다!")
else:
    print("   ⚠️ df_final이 아직 생성되지 않았습니다!")

디버깅: 데이터 확인

1. sample_stocks_with_sector 컬럼:
   ['InfoCode', 'DsCode', 'DsQtName', 'Exchange', 'ExchMnem', 'ExchIntCode', 'Ticker', 'ISIN', 'Cusip', 'Sedol', 'StatusCode', 'DelistDate', 'Region', 'sector', 'industry']
   sector 있음: True
   industry 있음: True

2. market_indices 상태:
   비어있음: False
   컬럼: ['date', 'spy_pi', 'spy_ri']
   레코드 수: 501

3. df_preprocessed 컬럼 (샘플 20개):
   ['InfoCode', 'DsCode', 'date', 'Ticker', 'ISIN', 'DsQtName', 'Exchange', 'ExchMnem', 'Region', 'sector', 'industry', 'open', 'high', 'low', 'close', 'volume', 'prev_open', 'prev_high', 'prev_low', 'prev_close']...
   Ticker 있음: True
   sector 있음: True

4. df_final 컬럼 (샘플 20개):
   전체 컬럼 수: 70
   ['InfoCode', 'DsCode', 'date', 'Ticker', 'ISIN', 'DsQtName', 'Exchange', 'ExchMnem', 'Region', 'sector', 'industry', 'open', 'high', 'low', 'close', 'volume', 'prev_open', 'prev_high', 'prev_low', 'prev_close']...

   ✓ 모든 예상 컬럼이 존재합니다!


In [12]:
df_final

,InfoCode,DsCode,date,Ticker,ISIN,DsQtName,Exchange,ExchMnem,Region,sector,...,return_20d,consecutive_up_days,spy_gap_pct,qqq_gap_pct,vix_level,vix_change,vix_category,market_gap_diff,sector_gap_pct,sector_relative_gap
0,73780,36248L,2024-01-09,None,None,FIRST TST.CAP.STRENGHT ETF,Nasdaq,NAS,US,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
1,73780,36248L,2024-01-09,None,None,FIRST TST.CAP.STRENGHT ETF,Nasdaq,NAS,US,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
2,73780,36248L,2024-01-09,None,None,FIRST TST.CAP.STRENGHT ETF,Nasdaq,NAS,US,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
3,73780,36248L,2024-01-10,None,None,FIRST TST.CAP.STRENGHT ETF,Nasdaq,NAS,US,NaN,...,NaN,1.0,0.566578,NaN,NaN,NaN,None,-0.679018,NaN,NaN
4,73780,36248L,2024-01-10,None,None,FIRST TST.CAP.STRENGHT ETF,Nasdaq,NAS,US,NaN,...,NaN,2.0,0.566578,NaN,NaN,NaN,None,-1.003703,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33594,385651,29023G,2025-12-18,None,None,OXLEY BRIDGE ACQUISITION A,Nasdaq,NAS,US,NaN,...,-0.397614,0.0,0.793464,NaN,NaN,NaN,None,-0.793464,NaN,NaN
33595,385651,29023G,2025-12-18,None,None,OXLEY BRIDGE ACQUISITION A,Nasdaq,NAS,US,NaN,...,-0.399602,0.0,0.793464,NaN,NaN,NaN,None,-0.791468,NaN,NaN
33596,385651,29023G,2025-12-18,None,None,OXLEY BRIDGE ACQUISITION A,Nasdaq,NAS,US,NaN,...,-0.496514,1.0,0.793464,NaN,NaN,NaN,None,-0.793464,NaN,NaN
33597,385651,29023G,2025-12-18,None,None,OXLEY BRIDGE ACQUISITION A,Nasdaq,NAS,US,NaN,...,-0.397614,0.0,0.793464,NaN,NaN,NaN,None,-0.793464,NaN,NaN


In [19]:
df_final.columns

Index(['InfoCode', 'DsCode', 'date', 'Ticker', 'ISIN', 'DsQtName', 'Exchange',
       'ExchMnem', 'Region', 'open', 'high', 'low', 'close', 'volume',
       'prev_open', 'prev_high', 'prev_low', 'prev_close', 'prev_volume',
       'gap_pct', 'is_gap_up', 'gap_size_category', 'intraday_return',
       'prev_return', 'prev_range_pct', 'prev_upper_shadow',
       'prev_lower_shadow', 'avg_volume_20d', 'volume_ratio', 'day_of_week',
       'month', 'quarter', 'year', 'is_month_start', 'is_month_end',
       'is_quarter_end', 'target_direction', 'target_return', 'market_cap',
       'StatusCode', 'DelistDate', 'ExchIntCode', 'ma_5', 'ma_20', 'ma_50',
       'above_ma5', 'above_ma20', 'above_ma50', 'ma5_ma20_cross', 'rsi_14',
       'rsi_category', 'atr_14', 'atr_ratio', 'bb_upper', 'bb_lower',
       'bb_middle', 'bollinger_position', 'return_5d', 'return_20d',
       'consecutive_up_days', 'spy_gap_pct', 'qqq_gap_pct', 'vix_level',
       'vix_change', 'vix_category', 'market_gap_diff', 's

In [11]:
df_final.to_csv("test_data.csv", encoding="utf-8")

### 5-6. 데이터 저장 (선택적)

In [16]:
# 샘플 데이터 저장 (선택적)
# 주석 제거하여 실행

save_path = "../data/processed/sample_preprocessed_df.parquet"
preprocessor.save_to_file(df_final, save_path)
print(f"✓ 데이터 저장 완료: {save_path}")

print("샘플 테스트 완료! 문제 없으면 전체 종목으로 확장 가능합니다.")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

---

## 다음 단계

### ✅ 완료된 것
1. **데이터 로더** - 종목 리스트, OHLCV, 섹터/산업, 시장 지수 로드
2. **전처리** - 갭 계산, 수익률, 거래량, 시간 Features (병렬 처리)
3. **기술적 지표** - MA, RSI, ATR, Bollinger Bands (병렬 처리)
4. **시장 컨텍스트** - SPY/QQQ/VIX 갭, 섹터별 갭

### 🔴 아직 구현 안 된 것
- 실적 발표 데이터 (earnings)
- 뉴스/센티먼트 데이터
- 애널리스트 레이팅 데이터
- 프리마켓 데이터 (Phase 2)

### 🚀 전체 종목 실행 방법
샘플 테스트가 성공하면, 위 코드에서:
- `sample_stocks = stock_list_full.head(10)` → `sample_stocks = stock_list_full`
- `start_date = datetime.now() - timedelta(days=365 * 2)` → `days=365 * 10` (10년)

### 📊 모델링 준비
- `df_final`을 사용하여 바로 모델 학습 가능
- 갭 상승 이벤트만 필터링: `df_final[df_final['is_gap_up'] == 1]`
- Train/Valid/Test 분할 후 모델 학습